# This notebook preprocesses all the raw data regarding services and combine them into one big dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
food_retailers = '/content/gdrive/My Drive/Colab Notebooks/CS506FinalProject/food_retailers_2017_pt.csv'
merged =  '/content/gdrive/My Drive/Colab Notebooks/CS506FinalProject/Final_merge.xlsx'

In [ ]:
import pandas as pd
import re
import requests
import urllib.parse
import json
import ast

#Open space:

In [ ]:
osdf = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CS506FinalProject/Open_Space_fully_processed.csv')
osdf.head(10)

,name,type,address,zip,lat,lon
0,Wilson Park,open space,Allston-Brighton,NaN,42.340985,-71.145930
1,Nickerson Field,open space,Allston-Brighton,NaN,42.353369,-71.119283
2,Smith Playground,open space,Allston-Brighton,NaN,42.364359,-71.131411
3,Fern Square,open space,Allston-Brighton,NaN,41.211271,-81.496505
4,Ringer Playground,open space,Allston-Brighton,NaN,42.350446,-71.138224
5,McKinney Playground,open space,Allston-Brighton,NaN,42.354751,-71.152922
6,Hobart Park,open space,Allston-Brighton,NaN,42.355735,-71.161270
7,Jackson Square,open space,Allston-Brighton,NaN,42.322596,-71.100257
8,Boyden Park,open space,Allston-Brighton,NaN,42.339545,-71.166317
9,Evergreen Cemetery,open space,Allston-Brighton,NaN,42.338336,-71.162957


#Food retailers (groceries)

In [ ]:
# do food retailers 
df = pd.read_csv(food_retailers)
df.head()

,objectid,name,address,address2,municipal,state,zipcode,recordtype,data_year,source,market_id,ebt,coupons,latitude,longitude,prim_type,shape
0,1,7-Eleven,462 Washington St,,Brighton,MA,2135,Unverified,2016,ReferenceUSA,0,,,42.348527,-71.158960,Convenience Stores,2400000001000000010004006A6900000C000000010000...
1,2,7-Eleven,131 Middlesex Tpke,,Burlington,MA,1803,Unverified,2016,ReferenceUSA,0,,,42.487965,-71.221743,Convenience Stores,2400000001000000010004006A6900000C000000010000...
2,3,7-Eleven,226 Winthrop Ave,,Lawrence,MA,1843,Unverified,2016,ReferenceUSA,0,,,42.686019,-71.139769,Convenience Stores,2400000001000000010004006A6900000C000000010000...
3,4,7-Eleven,1693 Route 6,,Somerset,MA,2726,Unverified,2016,ReferenceUSA,0,,,41.754381,-71.151866,Convenience Stores,2400000001000000010004006A6900000C000000010000...
4,5,7-Eleven,226 Broadway,,Taunton,MA,2780,Unverified,2016,ReferenceUSA,0,,,41.920467,-71.091320,Convenience Stores,2400000001000000010004006A6900000C000000010000...


Drop the unnecessary infomation.

In [ ]:
columns_drop = ['address2', 'state', 'recordtype', 'data_year', 'source', 'market_id', 'ebt', 'coupons', 'shape']
df = df.drop(columns = columns_drop)
df.head(5)

,objectid,name,address,municipal,zipcode,latitude,longitude,prim_type
0,1,7-Eleven,462 Washington St,Brighton,2135,42.348527,-71.158960,Convenience Stores
1,2,7-Eleven,131 Middlesex Tpke,Burlington,1803,42.487965,-71.221743,Convenience Stores
2,3,7-Eleven,226 Winthrop Ave,Lawrence,1843,42.686019,-71.139769,Convenience Stores
3,4,7-Eleven,1693 Route 6,Somerset,2726,41.754381,-71.151866,Convenience Stores
4,5,7-Eleven,226 Broadway,Taunton,2780,41.920467,-71.091320,Convenience Stores


Check how many cities are in the dataset:

In [ ]:
all_municipal = set(df['municipal'].to_list())
print(all_municipal)

{'Lunenburg', 'Raynham', 'Mission Hill', 'Wenham', 'West Springfield', 'South Chatham', 'Ware', 'Holland', 'South Deerfield', 'Jamaica Plain', 'Wendell', 'Webster', 'Mill River', 'Brookline', 'Hanson', 'Hudson', 'Topsfield', 'Bedford', 'Fall River', 'Ipswich', 'Chilmark', 'Menemsha', 'West Bridgewater', 'Lanesborough', 'West Boylston', 'Cuttyhunk', 'Wilkinsonville', 'Dartmouth', 'Feeding Hills', 'Eastham', 'Hatfield', 'Merrimac', 'Warren', 'Nahant', 'Tyngsboro', 'Danvers', 'Gloucester', 'Burlington', 'Leeds', 'Melrose', 'Woods Hole', 'North Carver', 'Lancaster', 'Westport Point', 'Gill', 'Osterville', 'Granby', 'Green Harbor', 'Framingham', 'Leverett', 'Bernardston', 'Newburyport', 'Dighton', 'Millbury', 'Lynn', 'Edgartown', 'Charlemont', 'Millville', 'Templeton', 'Hopkinton', 'North Adams', 'North Marshfield', 'Ludlow', 'North Easton', 'West Roxbury', 'Cohasset', 'Teaticket', 'Billerica', 'Chatham', 'South Easton', 'Thorndike', 'Blandford', 'Westwood', 'Brant Rock', 'Nantucket', 'Shir

Check Zipcodes:

In [ ]:
all_zipcode = set(df['zipcode'].to_list())
print(all_zipcode)

{2048, 0, 2050, 2051, 2052, 2053, 2054, 2056, 2059, 2061, 2062, 2066, 2067, 2072, 2081, 2090, 2093, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2134, 2135, 2136, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2148, 2149, 2150, 2151, 2152, 2155, 2163, 2169, 2170, 2171, 2176, 2180, 2184, 2186, 2187, 2188, 2189, 2190, 2191, 2199, 2201, 2203, 2210, 2215, 2222, 2301, 2302, 2322, 2324, 2330, 2332, 2333, 2334, 2338, 2339, 2341, 2343, 2345, 2346, 2347, 2351, 2355, 2356, 2359, 2360, 2364, 2367, 2368, 2370, 2375, 2379, 2381, 2382, 2420, 2421, 2445, 2446, 2451, 2452, 2453, 2458, 2459, 2460, 2461, 2464, 2465, 2466, 2467, 2468, 2472, 2474, 2476, 2478, 2481, 2482, 2492, 2493, 2494, 2532, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2543, 2552, 2553, 2554, 2556, 2557, 2558, 2559, 2561, 2562, 2563, 2564, 2568, 2571, 2574, 2575, 2576, 2601, 2630, 2631, 2632, 2633, 2635, 2638, 2639, 2641, 2642, 2643, 2645, 

Preprocess and drop unneeded instances.

In [ ]:
def is_city_Boston(zipcode):
  """zipcode: an integer number which is the zipcode"""
  """return true if it is within city of Boston; false otherwise"""
  zip = [2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 
         2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2163,
         2196, 2199, 2201, 2203, 2204, 2205, 2206, 2210, 2211, 2212, 2215, 2217, 2222, 2241, 2266, 
         2283, 2284, 2293, 2297, 2298]
  if zipcode in zip:
    return 1
  return 0

In [ ]:
all_prim_types = set(df['prim_type'].to_list())
print(all_prim_types)

{'Fruit & Vegetable Markets', 'Winter Markets', 'Department Stores (Except Discount Dept Stores)', 'Meat Markets', 'Fish & Seafood Markets', '452990', 'Supermarkets/Other Grocery (Exc Convenience) Strs', 'Farmers Markets', 'Warehouse Clubs & Supercenters', 'All Other Specialty Food Stores', 'Convenience Stores', 'Pharmacies & Drug Stores'}


The final data type should be:

'name', 'type', 'address', 'zip', 'lat', 'lon'

In [ ]:
columns_drop = ['objectid', 'municipal', 'prim_type']
df = df.drop(columns = columns_drop)
df = df.rename(columns = {'zipcode':'zip', 'latitude':'lat', 'longitude':'lon'})
df.insert(1, 'type', 'grocery')
df.head(5)

,name,type,address,zip,lat,lon
0,7-Eleven,grocery,462 Washington St,2135,42.348527,-71.158960
1,7-Eleven,grocery,131 Middlesex Tpke,1803,42.487965,-71.221743
2,7-Eleven,grocery,226 Winthrop Ave,1843,42.686019,-71.139769
3,7-Eleven,grocery,1693 Route 6,2726,41.754381,-71.151866
4,7-Eleven,grocery,226 Broadway,2780,41.920467,-71.091320


#Boston Business:
Load dataset:

In [ ]:
# do the last dataset
mdf = pd.read_excel(merged)
mdf.head(3)

,Unnamed: 0,LAT(Bing),LON(Bing),ADDRESS_LINE(Bing),BING_BUSINESS_NAME,BING_BUSINESS_TYPE,BING_BUSINESS_OTHER_TYPES,BING_FULL_ADDRESS,BING_LOCALITY,BING_ZIP_CODES,Google_place_id,Google_lat,Google_lon,Google_Business_name,Google_vicinity,Google_business_status,Google_business_types,Google_opening_hours,Google_price_level,Google_rating,Google_user_ratings_total,Google_reference,Google_viewport,Google_Zips,H distance,Fuzzy Similarity,Yelp_business_name,Yelp_categories,Yelp_id,Yelp_is_closed,Yelp_latitude,Yelp_locality,Yelp_longitude,Yelp_phone,Yelp_price,Yelp_rating,Yelp_review_count,Yelp_street_address,Yelp_Zip_Codes,UUID,All_addresses,All_Zips,All_Lat,All_Lon,locationName(Yellowpages),Yellowpages_categories,Yellowpages_locationAddress,Yellowpages_locationCity,Yellowpages_locationState,Yellowpages_locationZip,Yellowpages_locationPhone,Yellowpages_openTime,Years In Business(Yellowpages),Final_addresses,Final_Zips,Final_Lon,Final_Lat
0,0,42.342962,-71.068945,278 Shawmut Ave,Franklin Cafe,Café,"['Food and Drink', 'Bars', 'Restaurants', 'Cui...","278 Shawmut Ave, Boston, MA 02118, US",Boston,2118.0,ChIJt-MjhxN644kRFgBWz_vPOkM,42.342915,-71.068979,Franklin Cafe,"278 Shawmut Avenue, Boston",CLOSED_TEMPORARILY,"['bar', 'restaurant', 'food', 'point_of_intere...",NaN,2.0,4.5,225.0,ChIJt-MjhxN644kRFgBWz_vPOkM,"{'northeast': {'lat': 42.3443437802915, 'lng':...",2118.0,0.002410,100.0,Franklin Cafe,"['Lounges', 'American (New)']",fmw_0vST7GjdibObHWLOuQ,0.0,42.342910,"Boston, MA 02118",-71.068950,(617) 350-0010,$$,4.0,510.0,278 Shawmut Ave,2118.0,2bd8377e-884a-50aa-a190-ade425fd6ff3,Franklin Cafe,2118.0,42.342915,-71.068979,Franklin Cafe,"['Restaurants', 'American Restaurants', 'Bar &...",278 Shawmut Ave,Boston,MA,2118.0,(617) 350-0010,5:00 pm - 10:30 pm,24.0,Franklin Cafe,2118,-71.068979,42.342915
1,1,42.342103,-71.070354,314 Shawmut Ave,South End Buttery,Bakeries,"['Food and Drink', 'Bars', 'Restaurants', 'San...","314 Shawmut Ave, Boston, MA 02118, US",Boston,2118.0,ChIJZcElp3J644kRiDPnMMAYfbQ,42.342101,-71.070373,South End Buttery,"314 Shawmut Avenue, Boston",OPERATIONAL,"['restaurant', 'cafe', 'bar', 'food', 'point_o...",NaN,2.0,4.3,736.0,ChIJZcElp3J644kRiDPnMMAYfbQ,"{'northeast': {'lat': 42.3435140302915, 'lng':...",2118.0,0.006104,100.0,South End Buttery,"['Breakfast & Brunch', 'Bars', 'American (New)']",WR6RXp1I9QmO7-21lsqD8g,0.0,42.342109,"Boston, MA 02118",-71.070447,(617) 482-1015,$$,3.5,879.0,314 Shawmut Ave,2118.0,a1b22528-d444-5bbc-9baa-003a0143b608,South End Buttery,2118.0,42.342101,-71.070373,South End Buttery,"['Restaurants', 'American Restaurants', 'Take ...",314 Shawmut Ave,Boston,MA,2118.0,(617) 482-1015,6:00 am - 10:00 pm,9.0,South End Buttery,2118,-71.070373,42.342101
2,2,42.343681,-71.065255,117 E Berkeley St,JJ Foley's Cafe,Bars,"['Food and Drink', 'Restaurants', 'Retail', 'H...","117 E Berkeley St, Boston, MA 02118, US",Boston,2118.0,ChIJQw3LKG5644kRevjN1DTlXWE,42.343664,-71.065280,J.J. Foley's Cafe,"117 East Berkeley Street, Boston",OPERATIONAL,"['bar', 'restaurant', 'food', 'point_of_intere...",NaN,1.0,4.5,440.0,ChIJQw3LKG5644kRevjN1DTlXWE,"{'northeast': {'lat': 42.3450521802915, 'lng':...",2118.0,0.005026,100.0,JJ Foleys Cafe,['Irish Pub'],W62OJx5O8xwno-Dn3EuqJw,0.0,42.343689,"Boston, MA 02118",-71.065331,(617) 728-9101,$$,4.0,210.0,117 E Berkeley St,2118.0,d0267467-57cf-5c6c-8c0a-fbbe483a857d,J.J. Foley's Cafe,2118.0,42.343664,-71.065280,J.J. Foley's Cafe,"['Restaurants', 'Irish Restaurants', 'American...",117 E Berkeley St,Boston,MA,2118.0,(617) 728-9101,11:00 am - 11:00 am,111.0,J.J. Foley's Cafe,2118,-71.065280,42.343664


Drop all the unnecessary infomation. There are too many of them, so I do it in 2 steps.

In [ ]:
c_todrop = ['Unnamed: 0', 'BING_BUSINESS_OTHER_TYPES', 'BING_FULL_ADDRESS', 'Google_place_id', 'Google_opening_hours', 'Google_price_level', 'Google_rating', 'Google_user_ratings_total', 'Google_reference',
            'Google_viewport', 'H distance', 'Fuzzy Similarity', 'Yelp_id', 'Yelp_phone', 'Yelp_price', 'Yelp_rating', 'Yelp_review_count', 'UUID']
mdf = mdf.drop(columns = c_todrop)
mdf.head(3)

,LAT(Bing),LON(Bing),ADDRESS_LINE(Bing),BING_BUSINESS_NAME,BING_BUSINESS_TYPE,BING_LOCALITY,BING_ZIP_CODES,Google_lat,Google_lon,Google_Business_name,Google_vicinity,Google_business_status,Google_business_types,Google_Zips,Yelp_business_name,Yelp_categories,Yelp_is_closed,Yelp_latitude,Yelp_locality,Yelp_longitude,Yelp_street_address,Yelp_Zip_Codes,All_addresses,All_Zips,All_Lat,All_Lon,locationName(Yellowpages),Yellowpages_categories,Yellowpages_locationAddress,Yellowpages_locationCity,Yellowpages_locationState,Yellowpages_locationZip,Yellowpages_locationPhone,Yellowpages_openTime,Years In Business(Yellowpages),Final_addresses,Final_Zips,Final_Lon,Final_Lat
0,42.342962,-71.068945,278 Shawmut Ave,Franklin Cafe,Café,Boston,2118.0,42.342915,-71.068979,Franklin Cafe,"278 Shawmut Avenue, Boston",CLOSED_TEMPORARILY,"['bar', 'restaurant', 'food', 'point_of_intere...",2118.0,Franklin Cafe,"['Lounges', 'American (New)']",0.0,42.342910,"Boston, MA 02118",-71.068950,278 Shawmut Ave,2118.0,Franklin Cafe,2118.0,42.342915,-71.068979,Franklin Cafe,"['Restaurants', 'American Restaurants', 'Bar &...",278 Shawmut Ave,Boston,MA,2118.0,(617) 350-0010,5:00 pm - 10:30 pm,24.0,Franklin Cafe,2118,-71.068979,42.342915
1,42.342103,-71.070354,314 Shawmut Ave,South End Buttery,Bakeries,Boston,2118.0,42.342101,-71.070373,South End Buttery,"314 Shawmut Avenue, Boston",OPERATIONAL,"['restaurant', 'cafe', 'bar', 'food', 'point_o...",2118.0,South End Buttery,"['Breakfast & Brunch', 'Bars', 'American (New)']",0.0,42.342109,"Boston, MA 02118",-71.070447,314 Shawmut Ave,2118.0,South End Buttery,2118.0,42.342101,-71.070373,South End Buttery,"['Restaurants', 'American Restaurants', 'Take ...",314 Shawmut Ave,Boston,MA,2118.0,(617) 482-1015,6:00 am - 10:00 pm,9.0,South End Buttery,2118,-71.070373,42.342101
2,42.343681,-71.065255,117 E Berkeley St,JJ Foley's Cafe,Bars,Boston,2118.0,42.343664,-71.065280,J.J. Foley's Cafe,"117 East Berkeley Street, Boston",OPERATIONAL,"['bar', 'restaurant', 'food', 'point_of_intere...",2118.0,JJ Foleys Cafe,['Irish Pub'],0.0,42.343689,"Boston, MA 02118",-71.065331,117 E Berkeley St,2118.0,J.J. Foley's Cafe,2118.0,42.343664,-71.065280,J.J. Foley's Cafe,"['Restaurants', 'Irish Restaurants', 'American...",117 E Berkeley St,Boston,MA,2118.0,(617) 728-9101,11:00 am - 11:00 am,111.0,J.J. Foley's Cafe,2118,-71.065280,42.343664


In [ ]:
c_todrop = ['All_addresses', 'All_Zips', 'All_Lat', 'All_Lon', 'locationName(Yellowpages)', 'Yellowpages_categories', 'Yellowpages_locationAddress', 'Yellowpages_locationCity', 'Yellowpages_locationState',
            'Yellowpages_locationZip', 'Yellowpages_locationPhone', 'Yellowpages_openTime', 'Years In Business(Yellowpages)']
mdf = mdf.drop(columns = c_todrop)
mdf.head(3)

,LAT(Bing),LON(Bing),ADDRESS_LINE(Bing),BING_BUSINESS_NAME,BING_BUSINESS_TYPE,BING_LOCALITY,BING_ZIP_CODES,Google_lat,Google_lon,Google_Business_name,Google_vicinity,Google_business_status,Google_business_types,Google_Zips,Yelp_business_name,Yelp_categories,Yelp_is_closed,Yelp_latitude,Yelp_locality,Yelp_longitude,Yelp_street_address,Yelp_Zip_Codes,Final_addresses,Final_Zips,Final_Lon,Final_Lat
0,42.342962,-71.068945,278 Shawmut Ave,Franklin Cafe,Café,Boston,2118.0,42.342915,-71.068979,Franklin Cafe,"278 Shawmut Avenue, Boston",CLOSED_TEMPORARILY,"['bar', 'restaurant', 'food', 'point_of_intere...",2118.0,Franklin Cafe,"['Lounges', 'American (New)']",0.0,42.342910,"Boston, MA 02118",-71.068950,278 Shawmut Ave,2118.0,Franklin Cafe,2118,-71.068979,42.342915
1,42.342103,-71.070354,314 Shawmut Ave,South End Buttery,Bakeries,Boston,2118.0,42.342101,-71.070373,South End Buttery,"314 Shawmut Avenue, Boston",OPERATIONAL,"['restaurant', 'cafe', 'bar', 'food', 'point_o...",2118.0,South End Buttery,"['Breakfast & Brunch', 'Bars', 'American (New)']",0.0,42.342109,"Boston, MA 02118",-71.070447,314 Shawmut Ave,2118.0,South End Buttery,2118,-71.070373,42.342101
2,42.343681,-71.065255,117 E Berkeley St,JJ Foley's Cafe,Bars,Boston,2118.0,42.343664,-71.065280,J.J. Foley's Cafe,"117 East Berkeley Street, Boston",OPERATIONAL,"['bar', 'restaurant', 'food', 'point_of_intere...",2118.0,JJ Foleys Cafe,['Irish Pub'],0.0,42.343689,"Boston, MA 02118",-71.065331,117 E Berkeley St,2118.0,J.J. Foley's Cafe,2118,-71.065280,42.343664


### Find all the business types from Bing

In [ ]:
all_bing_business_types = list(set(mdf['BING_BUSINESS_TYPE'].to_list()))
all_bing_business_types = all_bing_business_types[1:] # remove the first nan
all_bing_business_types.sort()
print(all_bing_business_types)

['ATMs', 'Abuse and Addiction Treatment', 'Accounting and Bookkeeping', 'Acupuncturists', 'Adult Education', 'Adult Entertainment', 'Advertising Agencies', 'Afghani Restaurant', 'African Restaurant', 'Air Conditioning Services', 'Airline Ticket Offices', 'Airport Shuttles', 'Airports', 'Alcohol and Drug Treatment Centers', 'Allergists and Immunologists', 'Alternative Medicine', 'Alternative Schools', 'Ambulance and EMS Services', 'American Restaurant', 'Amusement Parks', 'Anesthesiologists', 'Animal Shelters and Humane Societies', 'Animal and Pet Services', 'Antique Stores', 'Apartments', 'Apparel', 'Apparel Accessories', 'Aquariums', 'Armed Forces', 'Art Galleries', 'Art Museums', 'Art Schools', 'Arts and Crafts Supply Stores', 'Arts and Entertainment', 'Assisted Living and Care Services', 'Associations', 'Attractions', 'Audio-Visual Production and Design', 'Audiologists', 'Australian Restaurant', 'Auto Body Shops', 'Auto Detailing Services', 'Auto Maintenance and Repair', 'Auto Parts

### Find all the business types from Google

In [ ]:
def sum_lists(lists):
  """lists: a 2d list"""
  """return a 1d list"""
  templist = []
  for i in lists:
    if type(i) != float: # nan
      i = re.sub("'", '', i)
      i_list = i.strip("[]").split(", ") # remove both , and whitespace
      templist += i_list
  return templist

temp = mdf['Google_business_types'].to_list()
all_google_business_types = list(set(sum_lists(temp)))
all_google_business_types.sort()
print(all_google_business_types)

['accounting', 'airport', 'aquarium', 'art_gallery', 'atm', 'bakery', 'bank', 'bar', 'beauty_salon', 'bicycle_store', 'book_store', 'bowling_alley', 'bus_station', 'cafe', 'campground', 'car_dealer', 'car_rental', 'car_repair', 'car_wash', 'cemetery', 'church', 'city_hall', 'clothing_store', 'convenience_store', 'courthouse', 'dentist', 'department_store', 'doctor', 'drugstore', 'electrician', 'electronics_store', 'embassy', 'establishment', 'finance', 'fire_station', 'florist', 'food', 'funeral_home', 'furniture_store', 'gas_station', 'general_contractor', 'grocery_or_supermarket', 'gym', 'hair_care', 'hardware_store', 'health', 'hindu_temple', 'home_goods_store', 'hospital', 'insurance_agency', 'jewelry_store', 'laundry', 'lawyer', 'library', 'light_rail_station', 'liquor_store', 'local_government_office', 'locality', 'locksmith', 'lodging', 'meal_delivery', 'meal_takeaway', 'mosque', 'movie_rental', 'movie_theater', 'moving_company', 'museum', 'natural_feature', 'neighborhood', 'nig

### Find all the business types from Yelp.

In [ ]:
temp = mdf['Yelp_categories'].to_list()
all_Yelp_categories	= list(set(sum_lists(temp)))
all_Yelp_categories.sort()
print(all_Yelp_categories)

['', '"Childrens Clothing"', '"Childrens Museums"', '"Mens Clothing"', '"Mens Hair Salons"', '"Womens Clothing"', '& Probates', '3D Printing', 'Acai Bowls', 'Accessories', 'Accountants', 'Acne Treatment', 'Active Life', 'Acupuncture', 'Adult', 'Adult Education', 'Adult Entertainment', 'Advertising', 'Aerial Fitness', 'Aestheticians', 'Afghan', 'African', 'Airlines', 'Airport Lounges', 'Airport Shuttles', 'Airport Terminals', 'Amateur Sports Teams', 'American (New)', 'American (Traditional)', 'Animal Shelters', 'Antiques', 'Apartment Agents', 'Apartments', 'Appliances', 'Appliances & Repair', 'Appraisal Services', 'Aquarium Services', 'Aquariums', 'Arcades', 'Architects', 'Art Classes', 'Art Galleries', 'Art Museums', 'Art Schools', 'Art Space Rentals', 'Art Supplies', 'Arts & Crafts', 'Arts & Entertainment', 'Asian Fusion', 'Assisted Living Facilities', 'Auction Houses', 'Audio/Visual Equipment Rental', 'Audiologist', 'Australian', 'Auto Customization', 'Auto Detailing', 'Auto Glass Se

### Manually Build all the dictionaries:

In [ ]:
Google_dic = {
    'hospital': ['hospital'],
    'healthcare': ['dentist', 'doctor', 'drugstore', 'health', 'pharmacy', 'physiotherapist'],
    'grocery': ['convenience_store', 'grocery_or_supermarket'],
    'supermarket': ['shopping_mall', 'supermarket'],
    'park': ['campground', 'park', 'rv_park', 'zoo']
}

In [ ]:
Bing_dic = {
    'hospital': ['Emergency Rooms', 'Hospitals', 'Medical Centers', 'Urgent Care'],
    'healthcare': ['Abuse and Addiction Treatment', 'Acupuncturists', 'Alcohol and Drug Treatment Centers', 
                   'Allergists and Immunologists', 'Alternative Medicine', 'Ambulance and EMS Services', 
                   'Anesthesiologists', 'Assisted Living and Care Services', 'Bariatric Medicine', 
                   'Cancer Treatment Centers', 'Cardiologists', 'Chiropractors', 'Dental Care', 'Dentists', 
                   'Dermatologists', 'Diagnostic Centers', 'Dialysis Clinics', 'Doctors', 
                   'Doctors and Clinics', 'Ear, Nose and Throat', 'Eating Disorder Treatment Centers', 
                   'Endocrinologists', 'Eye Care', 'Family and General Practice Physicians', 
                   'Gastroenterologists', 'Gynecologists', 'Health Clinics', 'Healthcare', 'Holistic Medicine',
                   'Hospices', 'Hypnosis And Hypnotherapy', 'Infectious Disease Specialists', 'Massage', 
                   'Medical Supply and Equipment Stores', 'Mental Health Services', 'Naturopathic Physicians', 
                   'Nephrologists', 'Neurologists', 'Nursing Homes and Life Care Facilities', 
                   'Nutritionists And Dietitians', 'Oncologists', 'Ophthalmologists', 
                   'Oral and Maxillofacial Surgeons', 'Orthodontists', 'Orthopedic Medicine', 
                   'Paternity Tests and Services', 'Pathologists', 'Pediatric Dentists', 
                   'Pediatricians', 'Pharmacies', 'Physical Therapists', 'Podiatrists', 'Psychiatrists', 
                   'Psychologists', 'Psychotherapists', 'Radiologists', 'Rehabilitation Services', 
                   'Rheumatologists', 'Sleep Disorder Centers', 'Speech Pathologists', 'Sports Medicine', 
                   'Urologists', 'Vitamin and Supplement Stores'],
    'grocery': ['Butcher Shop', 'Convenience Stores', 'Ethnic Food Stores', 'Farmers Markets', 'Fish Markets', 
                'Fruit and Vegetable Stands', 'Grocery'],
    'supermarket': ['Flea Markets', 'Malls and Shopping Centers', 'Outlet Stores', 'Supermarkets'],
    'park': ['Amusement Parks', 'Beaches', 'Botanical Gardens', 'Camps', 'Park and Rides', 
                            'Parks', 'Parks and Recreation', 'Playgrounds', 'Zoos']
}

Yelp_dic = {
    'hospital': ['Doctors', 'Emergency Rooms', 'Hospitals', 'Medical Centers', 'Personal Care Services', 'Urgent Care', 
                 'Walk-in Clinics'],
    'healthcare': ['Acne Treatment','Acupuncture', 'Behavior Analysts', 'Blood & Plasma Donation Centers', 'Cannabis Clinics', 
                   'Chiropractors', 'Concierge Medicine', 'Cosmetic Dentists', 'Cosmetic Surgeons', 'Counseling & Mental Health', 'Crisis Pregnancy Centers', 
                   'Cryotherapy', 'Dentists', 'Dermatologists', 'Diagnostic Imaging', 'Dietitians', 'Drugstores', 'Ear Nose & Throat', 'Endodontists', 
                   'General Dentistry', 'Health & Medical', 'Hepatologists', 'Hypnosis/Hypnotherapy', 'Internal Medicine', 'Laboratory Testing',
                   'Laser Eye Surgery/Lasik', 'Massage', 'Massage Therapy', 'Medical Supplies', 'Nutritionists', 'Obstetricians & Gynecologists', 
                   'Occupational Therapy', 'Oncologist', 'Ophthalmologists', 'Optometrists', 'Oral Surgeons', 'Orthodontists', 'Orthopedists', 
                   'Orthotics', 'Otologists', 'Pediatric Dentists', 'Pediatricians', 'Periodontists', 'Pharmacy', 'Physical Therapy', 'Plastic Surgeons', 
                   'Podiatrists', 'Prenatal/Perinatal Care', 'Prosthetics', 'Prosthodontists', 'Psychiatrists', 'Psychologists', 'Reflexology', 
                   'Rehabilitation Center', 'Sports Medicine', 'Traditional Chinese Medicine', 'Vitamins & Supplements'],   
    'grocery': ['Butcher', 'Cheese Shops','Chocolatiers & Shops', 'Convenience Stores', 'Farmers Market', 'Fruits & Veggies', 'Gastroenterologist', 'Grocery', 
                'International Grocery', 'Live/Raw Food', 'Meat Shops', 'Organic Stores', 'Seafood Markets'],
    'supermarket': ['Discount Store', 'Outlet Stores', 'Public Markets', 'Shopping', 'Shopping Centers'],
    'park': ['Beaches', 'Dog Parks', 'Parklets', 'Parks', 'Playgrounds', 'Skate Parks', 'Trampoline Parks','Baseball Fields', 'Basketball Courts']
    }

### I will use google info first; if it is not available for that entry, I will use Bing; Yelp is the last choice.

The final data type should be:

'Name', 'type', 'address', 'zip', 'lat', 'lon'

*In later experiment, we find there are some inconsistencies of business types from these 3 sources (e.g., they assign quite different types to the same instance). However, those cases are rare. This dataset is provided by the former SPARK! team, and we trust most of the infomation it provides is accurate. 

In [ ]:
def get_key(value, dic):
  """value: the unique value in the dictionary"""
  """dic: the dictionary"""
  """return: the key that contains the value"""
  for key, values in dic.items():
    if value in values:
      #print("key find!")
      return key
  return None

In [ ]:
def process_row(row):
  """take in a pandas row and return a list with values corresponding to description above"""
  r = []
  switch = -1 # 0 for Google, 1 for Bing and 2 for Yelp
  label = None
  # google
  types = row['Google_business_types']
  if type(types) != float:
    for typ in types:
      t = get_key(typ, Google_dic)
      if t != None:
        label = t
        break
    if label == None: # not an essential service
      return None
      #print("Google None")
    else:
      switch = 0
      #print("Google!")
  else: # Bing
    typ = row['BING_BUSINESS_TYPE']
    if type(typ) != float:
      label = get_key(typ, Bing_dic)
      if label == None: # not an essential service
        #print("Bing None")
        return None
      else:
        switch = 1
        #print("Bing!")
    else: # Yelp
      types = row['Yelp_categories'] # it is a string, for strange reason
      if type(types) != float:
        types = ast.literal_eval(types)
        for typename in types:
          t = get_key(typename, Yelp_dic)
          if t != None:
            label = t
            break
        if label == None: # not an essential service
          return None
        else:
          switch = 2
          #print("Yelp!")

  if switch == 0: # Google
    r.append(row['Google_Business_name']) # name
    r.append(label) # type
    #print("type")
    splited = row['Google_vicinity'].split(',')
    #print("split")
    r.append(splited[0]) # address
    #print("address")
    r.append(int(row['Google_Zips'])) # zip
    #print("zip")
    r.append(row['Google_lat']) # lat
    #print("lat")
    r.append(row['Google_lon']) # lon
    #print("lon")
  elif switch == 1: # Bing
    r.append(row['BING_BUSINESS_NAME']) # name
    r.append(label) # type
    #print("type")
    r.append(row['ADDRESS_LINE(Bing)']) # address
    #print("address")
    r.append(int(row['BING_ZIP_CODES'])) # zip
    #print("zip")
    r.append(row['LAT(Bing)']) # lat
    #print("lat")
    r.append(row['LON(Bing)']) # lon
    #print("lon")
  elif switch == 2: # Yelp
    r.append(row['Yelp_business_name']) # name
    r.append(label) # type
    #print("type")
    r.append(row['Yelp_street_address']) # address
    #print("address")
    r.append(int(row['Yelp_Zip_Codes'])) # zip
    #print("zip")
    r.append(row['Yelp_latitude']) # lat
    #print("lat")
    r.append(row['Yelp_longitude']) # lon
    #print("lon")
  else:
    return None
  return r

Do some tests:

In [ ]:
get_key('Seafood Markets', Yelp_dic)

'grocery'

In [ ]:
tttt = mdf.iloc[0]['Yelp_categories']
print(tttt)
print(type(tttt))

['Lounges', 'American (New)']
<class 'str'>


### Extract the useful services from the Business dataset:

In [ ]:
rows = []
for index, row in mdf.iterrows():
  r = process_row(row)
  if r != None:
    rows.append(r)

In [ ]:
print(len(rows))

3792


### Get the result:

Convert the lists into a dataframe:

In [ ]:
df1 = pd.DataFrame(rows, columns = ['name', 'type', 'address', 'zip', 'lat', 'lon'])

combine the food retailers and business:

In [ ]:
df2 = pd.concat([df, df1], ignore_index=True) # combine 2 dataframes (with same columns)

In [ ]:
df2.head(5)

,name,type,address,zip,lat,lon
0,7-Eleven,grocery,462 Washington St,2135,42.348527,-71.158960
1,7-Eleven,grocery,131 Middlesex Tpke,1803,42.487965,-71.221743
2,7-Eleven,grocery,226 Winthrop Ave,1843,42.686019,-71.139769
3,7-Eleven,grocery,1693 Route 6,2726,41.754381,-71.151866
4,7-Eleven,grocery,226 Broadway,2780,41.920467,-71.091320


remove those not in Boston City:

In [ ]:
df3 = df2[df2['zip'].map(lambda x: is_city_Boston(x)) == 1]

In [ ]:
df3.head(5)

,name,type,address,zip,lat,lon
0,7-Eleven,grocery,462 Washington St,2135,42.348527,-71.158960
22,24 Seven,grocery,655 Warren St,2121,42.309813,-71.082883
23,24 Seven Convenience Store,grocery,655 Warren St,2121,42.309813,-71.082883
29,Food Land,grocery,199 Adams St,2122,42.302039,-71.060150
35,660 Corp,grocery,543 Commonwealth Ave,2215,42.349045,-71.097122


Add the open space:

In [ ]:
df4 = pd.concat([df3, osdf], ignore_index=True)
df4.head()

,name,type,address,zip,lat,lon
0,7-Eleven,grocery,462 Washington St,2135.0,42.348527,-71.158960
1,24 Seven,grocery,655 Warren St,2121.0,42.309813,-71.082883
2,24 Seven Convenience Store,grocery,655 Warren St,2121.0,42.309813,-71.082883
3,Food Land,grocery,199 Adams St,2122.0,42.302039,-71.060150
4,660 Corp,grocery,543 Commonwealth Ave,2215.0,42.349045,-71.097122


In [ ]:
print(df4.shape)

(5620, 6)


Save the result:

In [ ]:
df4.to_csv('/content/gdrive/My Drive/Colab Notebooks/CS506FinalProject/merged_processed5.csv')